In [24]:
import pandas
import geopandas as gpd

In [25]:
# Merging Water and Census Datasets
wdf_g = gpd.read_file('data/water_quality_shp/water_quality_clean.shp')
wdf_g.crs = "EPSG:4326"
acs_g = gpd.read_file('data/geo_census.shp')
water_acs = gpd.sjoin(wdf_g, acs_g, how='inner', op='within')

In [28]:
water_acs.rename(mapper={'Date Sampl': 'date_sampled'}, 
                  axis='columns', inplace=True)
water_acs = water_acs.reset_index(drop=True)

In [33]:
water_acs_ltd = water_acs[['GEOID', 'date_sampled', '1st Draw', '2-3 Minute', '5 Minute',
                          'Threshold', 'geometry', 'tot_pop', 'tot_white', 
                          'tot_black', 'hh_size', 'med_income', 'occ_units',
                          'med_built_', 'med_rent', 'oo_hsng_un',
                          'perc_white', 'perc_non_w', 'perc_black', 'perc_owner']]
water_acs_ltd.head()

,GEOID,date_sampled,1st Draw,2-3 Minute,5 Minute,Threshold,geometry,tot_pop,tot_white,tot_black,hh_size,med_income,occ_units,med_built_,med_rent,oo_hsng_un,perc_white,perc_non_w,perc_black,perc_owner
0,170318330002,2020-01-26 07:42:00,1.0,1.0,1.0,0,POINT (-87.66359 41.88167),2013,1295,382,1.98,138650.0,1017,2000.0,2774.0,303,0.643318,0.356682,0.189767,0.297935
1,170318330002,2019-07-18 00:00:00,7.7,17.0,4.5,1,POINT (-87.65469 41.88235),2013,1295,382,1.98,138650.0,1017,2000.0,2774.0,303,0.643318,0.356682,0.189767,0.297935
2,170318330002,2019-09-16 07:54:00,1.0,1.0,1.0,0,POINT (-87.65468 41.88328),2013,1295,382,1.98,138650.0,1017,2000.0,2774.0,303,0.643318,0.356682,0.189767,0.297935
3,170318330002,2019-08-11 15:26:00,1.0,1.0,1.0,0,POINT (-87.65950 41.88302),2013,1295,382,1.98,138650.0,1017,2000.0,2774.0,303,0.643318,0.356682,0.189767,0.297935
4,170318330002,2018-08-19 22:56:00,1.0,1.0,1.0,0,POINT (-87.65950 41.88302),2013,1295,382,1.98,138650.0,1017,2000.0,2774.0,303,0.643318,0.356682,0.189767,0.297935


In [34]:
# Saving File
water_acs_ltd.to_file("data/merged.shp")

<ipython-input-34-6e553dfa39e3>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  water_acs_ltd.to_file("data/merged.shp")


In [ ]:
# Descriptive Statistics and Visualizations
